In [1]:
# Import dependencies
import pandas as pd
import requests
from config import api_key

In [5]:
# import yelpAPI
from yelpapi import YelpAPI
import argparse
from pprint import pprint

In [3]:
# Import top 200 cities in the US csv file
cities = pd.read_csv("Resources/top_200_cities.csv")
cities.head()

,rank,name,usps,pop2022,pop2010,growth,density,aland_sqmi
0,1,New York City,NY,8930002,8175133,0.0923,29729,300.381
1,2,Los Angeles,CA,3919973,3792621,0.0336,8359,468.956
2,3,Chicago,IL,2756546,2695598,0.0226,12124,227.369
3,4,Houston,TX,2345606,2099451,0.1172,3664,640.194
4,5,Phoenix,AZ,1640641,1445632,0.1349,3169,517.673


In [4]:
# Create list of top 200 cities
cities_list = cities["name"]
cities_list

0      New York City
1        Los Angeles
2            Chicago
3            Houston
4            Phoenix
           ...      
195             Kent
196        Fullerton
197           Denton
198          Visalia
199          McAllen
Name: name, Length: 200, dtype: object

In [20]:
with YelpAPI(api_key) as yelp_api:
    data = yelp_api.search_query(term='Breweries', location = 'New York City')
data

{'businesses': [{'id': 'bEFZtUHkpYzq9TI-N6k-OQ',
   'alias': 'evil-twin-brewing-dumbo-taproom-brooklyn',
   'name': 'Evil Twin Brewing - Dumbo Taproom',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/aDPcKzaBlAJXV3F2fxmXqg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/evil-twin-brewing-dumbo-taproom-brooklyn?adjust_creative=sC0soDHEztCO8fHkc2WZkA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=sC0soDHEztCO8fHkc2WZkA',
   'review_count': 51,
   'categories': [{'alias': 'breweries', 'title': 'Breweries'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.702834, 'longitude': -73.9905026},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '43 Main St',
    'address2': '',
    'address3': None,
    'city': 'Brooklyn',
    'zip_code': '11201',
    'country': 'US',
    'state': 'NY',
    'display_address': ['43 Main St', 'Brooklyn, NY 11201']},
   'phone': '',
   'display_phone': '',
   'distance': 419.9463732587953},

In [22]:
data['businesses'][0]['location']['city']

'Brooklyn'

In [23]:
# Create empty List ot hold all breweries
brewery_results = []

In [24]:
# Function to pull stores from yelp with location as the variable
def get_breweries(location):
    with YelpAPI(api_key) as yelp_api:
        # city_data is the json file with all businesses returned from search
        city_data = yelp_api.search_query(term='Breweries', location = location)
        # For each brewery in the search, return business name, coordinates, city, and buisness type
        for i in range(len(city_data['businesses'])):
            name = city_data['businesses'][i]['name']
            lat = city_data['businesses'][i]['coordinates']['latitude']
            long = city_data['businesses'][i]['coordinates']['longitude']
            city = city_data['businesses'][i]['location']['city']
            business_type = city_data['businesses'][i]['categories'][0]['alias']
            brewery_results.append({"Name":name, "Lat":lat, "Lng": long, "City":city, "Business Type":business_type})

In [25]:
# Apply get_breweries function to each location in cities_list
for city in cities_list:
    try:
        get_breweries(city)
    except:
        print(f"An exception occurred for {city}")

In [26]:
breweries_df = pd.DataFrame(brewery_results)
breweries_df.head()

,Name,Lat,Lng,City,Business Type
0,Evil Twin Brewing - Dumbo Taproom,40.702834,-73.990503,Brooklyn,breweries
1,Other Half Brewing - Domino Park,40.715957,-73.966897,Brooklyn,breweries
2,Evil Twin Brewing,40.696213,-73.904159,Ridgewood,breweries
3,Grimm Artisanal Ales,40.714256,-73.936579,Brooklyn,breweries
4,Transmitter Brewing,40.698559,-73.971204,Brooklyn,breweries


In [27]:
# Length of breweries_df
len(breweries_df)

3746